# Audio Preprocessing Pipeline

In [2]:
import librosa
import numpy as np
import noisereduce as nr
import webrtcvad
from IPython.display import Audio

AUDIO_PATH = r"C:\Users\Desktop\OneDrive\Desktop\smallest Small billgate.mp3"  # <-- change this later to your file
TARGET_SR = 16000               # 16 kHz

# 1. Audio preprocessing steps
- What happens:

The audio file (wav/mp3) is read and converted into a NumPy array of samples.

- Why needed:

ML models work on numerical data, not audio files.

In [3]:
audio, sr = librosa.load(AUDIO_PATH, sr=None)
print("Original sr:", sr)

Original sr: 44100


In [4]:
print("Before preprocessing:")
display(Audio(audio, rate=sr))

Before preprocessing:


# 2. Resampling
- What happens:

Converts audio from any sampling rate (e.g., 44.1kHz) to 16kHz.

- Why needed:

Models need all audio to have the same sampling rate for consistent features.

In [5]:
audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
sr = TARGET_SR
print("Resampled sr:", sr)

Resampled sr: 16000


# 3. Convert to mono
- What happens:

Stereo (2 channels) is converted to mono by averaging channels → 1 channel audio.

- Why needed:

ML models usually don’t need stereo; mono reduces size and simplifies training.

In [6]:
audio = librosa.to_mono(audio)
print("After mono shape:", audio.shape)

After mono shape: (71680,)


# 4. Normalization
- What happens:

Audio amplitude values are scaled to –1 to +1.

- Why needed:

Prevents loud audios from dominating training and keeps all samples consistent.

In [7]:
audio = librosa.util.normalize(audio)
print("After normalization, min/max:", audio.min(), audio.max())

After normalization, min/max: -0.8912603 1.0


# 5. Noise Reduction (optional but useful)
- What happens:

Background noise is removed using filtering (spectral gating).

- Why needed:

Clean audio improves feature quality → better model accuracy.

In [8]:
audio = nr.reduce_noise(y=audio, sr=sr)
print("After noise reduction shape:", audio.shape)

After noise reduction shape: (71680,)


# 6. Silence Removal
- What happens:

Leading and trailing silence sections are cut out.

- Why needed:

Silence adds no useful information and wastes training time.

In [9]:
audio_trimmed, index = librosa.effects.trim(audio, top_db=20)
print("Before trim length:", len(audio), "After trim length:", len(audio_trimmed))

# Use trimmed audio from now on
audio = audio_trimmed

Before trim length: 71680 After trim length: 70656


# 7. Voice Activity Detection
- What happens:

Removes non-speech parts (breaths, noise, music, silence).

Keeps only actual speaking segments.

- Why needed:

Models should learn from speech only → improves training.

In [10]:
def apply_webrtc_vad(audio, sr=16000, aggressiveness=2):

    if sr != 16000:
        raise ValueError("WebRTC VAD example expects 16kHz sample rate.")

    vad = webrtcvad.Vad(aggressiveness)  # 0–3 (3 = most aggressive)

    # Convert float32 [-1, 1] -> int16 PCM bytes
    int16_audio = (audio * 32767).astype(np.int16)
    pcm_bytes = int16_audio.tobytes()

    frame_ms = 30  # 30 ms frames
    frame_size = int(sr * frame_ms / 1000)      # samples per frame
    bytes_per_frame = frame_size * 2           # int16 = 2 bytes

    voiced_samples = []

    for start in range(0, len(pcm_bytes), bytes_per_frame):
        frame = pcm_bytes[start:start+bytes_per_frame]
        if len(frame) < bytes_per_frame:
            break
        if vad.is_speech(frame, sr):
            # keep only speech frames
            voiced_samples.append(np.frombuffer(frame, dtype=np.int16))

    if len(voiced_samples) == 0:
        print("VAD removed everything, returning original audio.")
        return audio

    voiced_int16 = np.concatenate(voiced_samples)
    voiced_float = voiced_int16.astype(np.float32) / 32767.0
    return voiced_float

# Apply VAD
audio_vad = apply_webrtc_vad(audio, sr=sr)
print("After VAD length:", len(audio_vad))

# Use VAD audio from now on
audio = audio_vad


After VAD length: 65280


# 8. Framing/Windowing
- What happens:

Audio is divided into small overlapping chunks:

    25 ms frames

    10 ms hop

    Hamming window smooths edges

- Why needed:

Speech changes every few milliseconds.

ML features (MFCC/Log-Mel) are computed per frame.

In [11]:
frame_length_ms = 25
hop_length_ms   = 10

n_fft = int(sr * frame_length_ms / 1000)   # samples per frame
hop_length = int(sr * hop_length_ms / 1000)

# 9. Feature Extraction
- What happens:

Raw audio is converted into meaningful patterns:

    MFCC → represents how humans hear sound

    Log-Mel → time-frequency representation

- Why needed:

ML/DL models perform much better on features than raw waveforms.

In [12]:
# Example: MFCC
mfcc = librosa.feature.mfcc(y=audio, sr=sr,
                            n_mfcc=13,
                            n_fft=n_fft,
                            hop_length=hop_length,
                            window='hamming')
print("MFCC shape:", mfcc.shape)

# Example: Log-Mel Spectrogram
mel = librosa.feature.melspectrogram(y=audio, sr=sr,
                                     n_fft=n_fft,
                                     hop_length=hop_length,
                                     window='hamming')
log_mel = librosa.power_to_db(mel)
print("Log-Mel shape:", log_mel.shape)

MFCC shape: (13, 409)
Log-Mel shape: (128, 409)


# 10. Padding or Trimming to Fixed Length (on waveform)
- What happens:

Every audio clip is forced to the same duration (e.g., 3 seconds).

- Why needed:

Neural networks require fixed-size input shapes.

In [13]:
#   here: 3 seconds -> 16000 * 3 samples
fixed_length = sr * 3  # 3 seconds at 16kHz
audio_fixed = librosa.util.fix_length(audio, size=fixed_length)
print("Fixed waveform length:", len(audio_fixed))

Fixed waveform length: 48000


# 11. Data Augmentation (for training)
- What happens:

We create new “modified” versions of audio:

    time-stretch

    pitch shift

    background noise

    random crop

- Why needed:

Makes the model more robust.

Prevents overfitting on small datasets.

In [14]:
def random_crop(audio, sr=16000, sec=3):
    target = int(sr * sec)
    if len(audio) <= target:
        return librosa.util.fix_length(data=audio, size=target)
    start = np.random.randint(0, len(audio) - target)
    return audio[start:start + target]

def add_noise(audio, level=0.005):
    return audio + level * np.random.randn(len(audio)).astype(audio.dtype)

def safe_time_stretch(audio, sr, low=0.9, high=1.1):
    try:
        rate = float(np.random.uniform(low, high))
        return librosa.effects.time_stretch(y=audio, rate=rate)
    except Exception as e:
        warnings.warn(f"time_stretch failed: {e}; skipping stretch")
        return audio

def safe_pitch_shift(audio, sr, min_steps=-2, max_steps=2):
    try:
        steps = float(np.random.uniform(min_steps, max_steps))
        return librosa.effects.pitch_shift(y=audio, sr=sr, n_steps=steps)
    except Exception as e:
        warnings.warn(f"pitch_shift failed: {e}; skipping pitch shift")
        return audio

def simple_augment_safe(audio, sr=16000, sec=3):
    # 1) ensure float32 and normalized
    audio = np.asarray(audio, dtype=np.float32)
    if audio.dtype != np.float32:
        audio = audio.astype(np.float32)
    # optional: clip to [-1,1]
    audio = np.clip(audio, -1.0, 1.0)

    # 2) crop FIRST to keep transforms small (prevents kernel death)
    audio = random_crop(audio, sr, sec)

    # 3) apply transforms on short audio, with safe fallbacks
    if np.random.rand() < 0.5:
        audio = safe_time_stretch(audio, sr)

    if np.random.rand() < 0.5:
        audio = safe_pitch_shift(audio, sr)

    if np.random.rand() < 0.5:
        audio = add_noise(audio, level=0.005)

    # 4) final crop/fix to exact length
    return random_crop(audio, sr, sec)

# Usage:
audio_aug = simple_augment_safe(audio_fixed, sr)
print("Augmented length (samples):", len(audio_aug))

Augmented length (samples): 48000


In [15]:
print("After preprocessing:")
display(Audio(audio_aug, rate=sr))

After preprocessing:
